In [ ]:
import os

import functions_framework

from google.cloud import bigquery
from google.cloud import storage

codigo juan 

In [ ]:


Instatiate clients
cloud_storage = storage.Client()
bq = bigquery.Client()

project_id = 'dms-pfh'

Triggered by a change in a storage bucket
@functions_framework.cloud_event
def load_from_json(cloud_event):

    data = cloud_event.data

    event_id = cloud_event["id"]
    event_type = cloud_event["type"]

    print(f'Cloud event ID: {event_id}')
    print(f'Event type: {event_type}')

    # 1st. Get the file URI
    bucket = data["bucket"]
    name = data["name"]
    # Create URI for ingestion
    file_uri = f'gs://{bucket}/{name}'
---------------------------------------

---------------------------------------
    # Insert into Bigquery ----
    # Create a table id
    table_id = f'{project_id}.raw_revs.reviews'

    # Create job configurations
    job_config = bigquery.LoadJobConfig(
      autodetect=True,
      source_format='NEWLINE_DELIMITED_JSON'
    )

    # Requesting the job
    load_job = bq.load_table_from_uri(
      file_uri,
      table_id,
      location='us-central1',
      job_config=job_config
    )

    load_job.result()

    destination_table = bq.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

filtrado por la mascara

In [60]:
table_id=r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\REVIEW-ESTADOS\reviews-estados\review-Alabama\1.json"


def ETL_reviews(df_json):
    df=df_json.copy()
    #eliminado de columnas PICS y RESP
    df.drop(columns=["pics", "resp"], inplace=True)
    # Convertir la columna 'time' a formato de fecha y hora
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    # Proceso de ETL
    
    for columna in df.columns:
        if df[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df[columna] = df[columna].fillna("No data").str.strip()
        elif df[columna].dtype in ['int64', 'float64']:
        # Si la columna es numérica, llenar los valores nulos con 0
            df[columna] = df[columna].fillna(0)
        
    # Eliminar duplicados basados en las columnas 'gmap_id' y 'user_id'
    df.drop_duplicates(subset=['gmap_id', 'user_id'], inplace=True)
    #cambiar tipo dato columna user_id a string
    df['user_id'] = df['user_id'].astype(str)
    
    return df


def filtracion(df_clean, df_mascara):
    # Obtener los gmap_id del DataFrame df_final
    gmap_ids_filtrar = df_mascara['gmap_id']

    # Filtrar el DataFrame df_clean utilizando los gmap_id obtenidos
    df_reviews = df_clean[df_clean['gmap_id'].isin(gmap_ids_filtrar)]

    # Restablecer los índices del DataFrame resultante
    df_reviews.reset_index(drop=True, inplace=True)
    return (df_reviews)


#table_id = ruta
df=pd.read_json(table_id, lines=True)
df_mascara=pd.read_csv("DIRECCION alojamiento DEL BUCKET DE LA MASCARA")
df_clean=ETL_reviews(df)
df_filtrado=filtracion(df_clean, df_mascara)

codigo para la cracion de la mascara

In [ ]:
import pandas as pd


def creacion_mascara():

    todas_mascaras = []

    # Iterar sobre los nombres de archivo del 1 al 11 inclusive
    for i in range(1, 12):
        # Construir la ruta del archivo
        ruta = fr"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\GOOGLE\DATA SITIOS\metadata-sitios\{i}.json"
        
        # Leer el archivo JSON
        data = pd.read_json(ruta, lines=True)
        
        # Filtrar el DataFrame para las categorías permitidas
        df_mascara = data[data['category'].apply(lambda x: isinstance(x, list) and any(cat in ['Restaurant', 'Bar', 'Pub', 'Cafe'] for cat in x))]
        
        # Seleccionar la columna 'gmap_id' del DataFrame filtrado
        df_mascara = df_mascara[['gmap_id']]
        
        # Eliminar los valores duplicados de la columna 'gmap_id'
        df_mascara_sin_duplicados = df_mascara.drop_duplicates(subset=['gmap_id'])
        
        # Agregar la máscara sin duplicados a la lista de todas las máscaras
        todas_mascaras.append(df_mascara_sin_duplicados)

    # Combinar todas las máscaras en un solo DataFrame
    df_final = pd.concat(todas_mascaras, ignore_index=True)
    return (df_final)

mascara=creacion_mascara()
mascara.to_csv("mascara.csv", index=False)
#aca se guardaria en el bucket

In [ ]:
import pandas as pd
from google.cloud import storage

# Instatiate clients
cloud_storage = storage.Client()

project_id = 'dms-pfh'

# Crear función para la creación de la máscara
def creacion_mascara():
    todas_mascaras = []

    # Iterar sobre los nombres de archivo del 1 al 11 inclusive
    for i in range(1, 12):
        # Construir la ruta del archivo
        ruta = fr"gs://data_lake_pf_henry/Google Maps/metadata-sitios/{i}.json"
        
        # Leer el archivo JSON
        data = pd.read_json(ruta, lines=True)
        
        # Filtrar el DataFrame para las categorías permitidas
        df_mascara = data[data['category'].apply(lambda x: isinstance(x, list) and any(cat in ['Restaurant', 'Bar', 'Pub', 'Cafe'] for cat in x))]
        
        # Seleccionar la columna 'gmap_id' del DataFrame filtrado
        df_mascara = df_mascara[['gmap_id']]
        
        # Eliminar los valores duplicados de la columna 'gmap_id'
        df_mascara_sin_duplicados = df_mascara.drop_duplicates(subset=['gmap_id'])
        
        # Agregar la máscara sin duplicados a la lista de todas las máscaras
        todas_mascaras.append(df_mascara_sin_duplicados)

    # Combinar todas las máscaras en un solo DataFrame
    df_final = pd.concat(todas_mascaras, ignore_index=True)
    return df_final

# Crear la máscara de datos
mascara = creacion_mascara()

# Guardar la máscara en un archivo CSV en un bucket de Cloud Storage
bucket_name = 'data_lake_pf_henry'
blob_name = 'mascara.csv'
bucket = cloud_storage.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Convertir la máscara a formato CSV y guardarla en el blob
mascara_csv = mascara.to_csv("mascara.csv",index=False)
blob.upload_from_string(mascara_csv, content_type='text/csv')

print(f"Máscara guardada correctamente en gs://{bucket_name}/{blob_name}")




In [ ]:

import pandas as pd
from google.cloud import storage, bigquery
from google.cloud.storage import Blob
from google.cloud import functions_framework

# Instatiate clients
cloud_storage = storage.Client()
bq = bigquery.Client()

project_id = 'dms-pfh'

@functions_framework.cloud_event
def load_from_json(cloud_event):
    # Get the file URI
    data = cloud_event.data
    bucket = data["bucket"]
    name = data["name"]
    file_uri = f'gs://{bucket}/{name}'

    # Load JSON data from Cloud Storage
    df = pd.read_json(file_uri, lines=True)
    
    # Clean and filter the data
    df_clean = ETL_reviews(df)
    df_filtered = filtracion(df_clean)
    
    # Save the filtered data back to Cloud Storage
    bucket_name = 'bucket_name'
    blob_name = 'mascara.csv'
    blob = Blob(blob_name, cloud_storage.bucket(bucket_name))
    df_filtered.to_csv(f"gs://{bucket_name}/{blob_name}", index=False)
    
    # Load data into BigQuery if needed
    load_data_into_bigquery(df_filtered)


def ETL_reviews(df):
    # Your ETL process here
    df=df.copy()
    #eliminado de columnas PICS y RESP
    df.drop(columns=["pics", "resp"], inplace=True)
    # Convertir la columna 'time' a formato de fecha y hora
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    # Proceso de ETL
    
    for columna in df.columns:
        if df[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df[columna] = df[columna].fillna("No data").str.strip()
        elif df[columna].dtype in ['int64', 'float64']:
        # Si la columna es numérica, llenar los valores nulos con 0
            df[columna] = df[columna].fillna(0)
        
    # Eliminar duplicados basados en las columnas 'gmap_id' y 'user_id'
    df.drop_duplicates(subset=['gmap_id', 'user_id'], inplace=True)
    #cambiar tipo dato columna user_id a string
    df['user_id'] = df['user_id'].astype(str)
    
    return df


def filtracion(df_clean):
    # Obtener los gmap_id del DataFrame df_final
    df_mascara=pd.read_csv("gs://data_lake_pf_henry/Google Maps/mascara.csv")
    gmap_ids_filtrar = df_mascara['gmap_id']

    # Filtrar el DataFrame df_clean utilizando los gmap_id obtenidos
    df_reviews = df_clean[df_clean['gmap_id'].isin(gmap_ids_filtrar)]

    # Restablecer los índices del DataFrame resultante
    df_reviews.reset_index(drop=True, inplace=True)
    return df_reviews


def load_data_into_bigquery(df):
    # Insert data into BigQuery
    table_id = f'{project_id}.raw_revs.reviews'
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )
    job = bq.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print("Loaded {} rows into BigQuery table {}".format(job.output_rows, table_id))

if __name__ == "__main__":
    load_from_json({})



In [ ]:
import functions_framework
#import os
import pandas as pd
from google.cloud import storage
from google.cloud.functions import storage
# Triggered by a change in a storage bucket
@functions_framework.cloud_event


# Instatiate clients
cloud_storage = storage.Client()

project_id = 'dms-pfh'

# Crear función para la creación de la máscara
def creacion_mascara(event):
    # 1. Obtener el nombre del bucket y el nombre del archivo
    bucket_name = event['bucket']
    file_name = event['name']

    # 2. Verificar si el evento corresponde a un archivo CSV
    if file_name.endswith('.csv'):
        # 3. Leer el archivo CSV
        blob = cloud_storage.bucket(bucket_name).blob(file_name)
        content = blob.download_as_string()
        df = pd.read_csv(content)

        # 4. Filtrar el DataFrame para las categorías permitidas
        df_mascara = df[df['category'].apply(lambda x: isinstance(x, list) and any(cat in ['Restaurant', 'Bar', 'Pub', 'Cafe'] for cat in x))]
        
        # 5. Seleccionar la columna 'gmap_id' del DataFrame filtrado
        df_mascara = df_mascara[['gmap_id']]
        
        # 6. Eliminar los valores duplicados de la columna 'gmap_id'
        df_mascara_sin_duplicados = df_mascara.drop_duplicates(subset=['gmap_id'])

        # 7. Guardar la máscara en un archivo CSV en el mismo bucket
        output_bucket = cloud_storage.bucket(bucket_name)
        output_blob = output_bucket.blob('mascara.csv')
        output_blob.upload_from_string(df_mascara_sin_duplicados.to_csv(index=False), content_type='text/csv')

        print(f"Máscara guardada correctamente en gs://{bucket_name}/mascara.csv")

# Establecer las credenciales de autenticación si es necesario
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ruta_a_tus_credenciales.json"

if __name__ == "__main__":
    creacion_mascara({})


nico codigo

In [ ]:
import functions_framework
import pandas as pd
from google.cloud import storage
import json
import logging
from pandas_gbq import to_gbq

@functions_framework.http
def ETL_reviews_bigquery(request):
    try:
        # Obtén la carpeta en el bucket desde los parámetros de la URL
        estado = request.args.get('estado')

        # Verifica si se proporcionó la carpeta en la URL
        if not estado:
            return "Error: El parámetro 'estado' es necesario en la URL", 400

        # Define el nombre del bucket y la ruta de la carpeta temporal
        bucket_name = 'data_lake_pf_henry'
        bucket_path = f'Google Maps/reviews-estados/{estado}/'
        temporal_folder_path = 'temporal/'

        # Crea un cliente de Cloud Storage
        storage_client = storage.Client()

        # Obtiene el bucket
        bucket = storage_client.bucket(bucket_name)

        # Lista todos los blobs en la carpeta
        blobs = bucket.list_blobs(prefix=bucket_path)

        # Inicializa una lista para almacenar los DataFrames de cada archivo
        dfs = []

        # Itera sobre cada archivo en la carpeta
        for blob in blobs:
            # Descarga el contenido del blob como un string
            json_string = blob.download_as_text()

            # Dividir el string por nuevas líneas y cargar cada objeto JSON
            json_objects = [json.loads(j) for j in json_string.splitlines()]

            # Convierte la lista de objetos JSON en un DataFrame de Pandas
            df = pd.DataFrame(json_objects)
            dfs.append(df)

        # Concatena todos los DataFrames en uno solo
        df_combined = pd.concat(dfs)

        # Guarda el DataFrame combinado como un archivo CSV en Cloud Storage en la carpeta temporal
        csv_file_path = 'gs://{}/{}'.format(bucket_name, temporal_folder_path + 'combined_data.csv')
        df_combined.to_csv(csv_file_path, index=False)

        # Carga el archivo CSV en BigQuery
        to_gbq(df_combined, destination_table='proyecto-final-henry-412703.Datawarehouse_PF.review-Alaska', if_exists='replace')

        # Devuelve la respuesta HTTP con el JSON del DataFrame
        return "Carga exitosa!", 200
    except Exception as e:
        logging.error(f"Error en la función: {e}", exc_info=True)
        return f"Error: {e}", 500
